In [16]:
from matplotlib import pyplot as plt
import torch
from torchvision import datasets, transforms, models
import torch.nn.functional as F
from torch import nn 
from torch import optim
from tqdm import tqdm_notebook
%matplotlib inline

In [17]:
train_transform = transforms.Compose([transforms.RandomRotation(30),
                                      transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])])
# test_transform = transforms.Compose([transforms.Resize(255),
#                                      transforms.CenterCrop(224),
#                                      transforms.ToTensor(),
#                                      transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])])

In [18]:
batch_size = 32
train_data = datasets.ImageFolder('../input/images/Images/', transform=train_transform)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)


In [19]:
model = models.vgg19(pretrained=True)
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU(inplace)

In [20]:
for param in model.parameters():
    param.requires_grad = False

**Create Classifier**

In [21]:
from collections import OrderedDict
classifier = nn.Sequential(
    OrderedDict([
        ('fc1', nn.Linear(25088, 512)),
        ('relu', nn.ReLU()),
        ('fc2', nn.Linear(512, 120)),
        ('output', nn.LogSoftmax(dim=1))]))
model.classifier = classifier

In [23]:
# epoch
epoch = 10
# loss function
criterion = nn.NLLLoss()

# optimizer
optimizer = optim.Adam(model.classifier.parameters(), lr=0.01)

# move model to cuda
model.to('cuda')
for e in range(epoch):
    train_loss = 0
    test_loss = 0
    # train model
    for inputs, labels in tqdm_notebook(train_loader):
        inputs, labels = inputs.to('cuda'), labels.to('cuda')
        # make gradient zero
        torch.no_grad()
        outputs = model(inputs)
        # calculate loss
        loss = criterion(outputs, labels)
        # backward pass
        loss.backward()
        # optimize gradients
        optimizer.step()
        train_loss += loss.item()                
    # average loss
    train_loss /= len(train_loader.dataset)
    print("Train loss:", train_loss)

Train loss: 1.3453318207789904


Train loss: 0.2476321296858718


Train loss: 0.2635742334397356


Train loss: 0.22569520929927844


Train loss: 0.23807431256689993


Train loss: 0.27892303596548484


Train loss: 0.26487085928143056


Train loss: 0.2867319351614275


Train loss: 0.2760083898858496


Train loss: 0.2886689057271381
